# Comparing the perplexity of different language models - ALBERT and GPT-2

### Import libraries

In [1]:
#!pip install torch transformers datasets
#!pip install ipywidgets
#!pip install sentencepiece
#!pip install evaluate

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string

import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from tqdm import tqdm

import random

In [3]:
news_df = pd.read_csv('bbc_news.csv')
news_df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [4]:
news_df['text'] = news_df['title'] + ' ' + news_df['description']
news_df = news_df[['pubDate','text']]
news_df['pubDate'] = pd.to_datetime(news_df['pubDate'])
news_df['text'] = news_df['text'].astype(str).str.lower()
news_df.head()

,pubDate,text
0,2022-03-07 08:01:56,ukraine: angry zelensky vows to punish russian...
1,2022-03-06 22:49:58,war in ukraine: taking cover in a town under a...
2,2022-03-07 00:14:42,ukraine war 'catastrophic for global food' one...
3,2022-03-07 00:05:40,manchester arena bombing: saffie roussos's par...
4,2022-03-07 08:15:53,ukraine conflict: oil price soars to highest l...


In [5]:
news_df.shape

(41780, 2)

In [6]:
# Split data into train and remaining (val+test)
train_size = 0.6
val_size = 0.2


train_data, remaining_data = train_test_split(news_df, test_size=(1 - train_size), random_state=42)
val_data, test_data = train_test_split(remaining_data, test_size=(val_size / (1 - train_size)), random_state=42)


print(f"Train size: {len(train_data)}")
print(f"Val size: {len(val_data)}")
print(f"Test size: {len(test_data)}")

Train size: 25068
Val size: 8356
Test size: 8356


In [7]:
# Preprocess Dataset
def preprocess_nsp_gpt2(dataset, tokenizer, max_length=128):
    """
    Prepares the dataset for NSP with GPT-2.
    Positive examples: Sequential sentences.
    Negative examples: Random sentence pairs.
    """
    inputs, labels = [], []
    dataset = dataset.dropna(subset=["text"])

    for article in dataset["text"]:
        sentences = article.split(". ")  # Split into sentences.
        for i in range(len(sentences) - 1):
            # Positive example: Consecutive sentences.
            inputs.append((sentences[i], sentences[i + 1]))
            labels.append(1)

            # Negative example: Random pair.
            rand_index = np.random.randint(0, len(sentences))
            inputs.append((sentences[i], sentences[rand_index]))
            labels.append(0)

    encodings = tokenizer(
        [x[0] + " " + x[1] for x in inputs],
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    encodings["labels"] = torch.tensor(labels)
    return encodings

In [8]:
class NSPDataset(Dataset):
    def __init__(self, tokenized_data):
        """
        Args:
            tokenized_data: Dictionary with tokenized inputs and labels.
        """
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]
        self.labels = tokenized_data["labels"]

    def __len__(self):
        return self.input_ids.shape[0]

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

In [9]:
# Load ALBERT tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [10]:
# Ensure the tokenizer can handle sequences with special tokens
tokenizer.pad_token = tokenizer.eos_token

In [11]:
# Preprocess datasets
max_length = 128
tokenized_train = preprocess_nsp_gpt2(train_data, tokenizer, max_length=max_length)
tokenized_val = preprocess_nsp_gpt2(val_data, tokenizer, max_length=max_length)
tokenized_test = preprocess_nsp_gpt2(test_data, tokenizer, max_length=max_length)

train_dataset = NSPDataset(tokenized_train)
val_dataset = NSPDataset(tokenized_val)
test_dataset = NSPDataset(tokenized_test)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Use Untrained ALBERT for NSP

In [12]:
untrained_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
# Ensure the model recognizes the pad token
untrained_model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
untrained_model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [13]:
# Evaluation Function
def evaluate_gpt2_model(model, dataloader):
    """
    Evaluate GPT-2 on NSP and calculate:
    - Accuracy
    - Perplexity
    """
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0
    total_batches = 0

    for batch in tqdm(dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = outputs.loss

            # Aggregate loss
            total_loss += loss.item()
            total_batches += 1

            # Get predictions
            preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / total_batches
    perplexity = np.exp(avg_loss)

    return {
        "accuracy": accuracy,
        "perplexity": perplexity,
    }


# Compute Metrics
metrics = evaluate_gpt2_model(untrained_model, test_dataloader)
print(f"Evaluation Results for Model: {metrics}")


Evaluating: 100%|██████████| 22/22 [00:05<00:00,  3.95it/s]

Evaluation Results for Model: {'accuracy': 0.5114613180515759, 'perplexity': 26.640778211982646}


In [14]:
# Clear any unused GPU memory
torch.cuda.empty_cache()

Training ALBERT for NSP

In [15]:
# Step 2: Fine-Tune ALBERT for NSP
trained_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
# Ensure the model recognizes the pad token
trained_model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
trained_model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [16]:
optimizer = torch.optim.AdamW(trained_model.parameters(), lr=2e-5)

# Training Loop
def train_gpt2_model(model, train_dataloader, val_dataloader, epochs=3):
    """
    Fine-tunes GPT-2 on the NSP task.
    """
    best_val_loss = float("inf")

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0.0

        for batch in tqdm(train_dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss:.4f}")

        # Evaluate on validation set
        val_metrics = evaluate_gpt2_model(model, val_dataloader)
        print(f"Validation Metrics: {val_metrics}")

        # Save the best model based on validation perplexity
        if val_metrics["perplexity"] < best_val_loss:
            best_val_loss = val_metrics["perplexity"]
            torch.save(model.state_dict(), "best_gpt2_nsp_model.pt")
            print("Saved best model!")

    print("Training Complete!")
    return model

In [17]:
# Train the model
trained_gpt2_model = train_gpt2_model(trained_model, train_dataloader, val_dataloader)

Epoch 1/3


Training: 100%|██████████| 69/69 [04:19<00:00,  3.76s/it]


Training Loss: 0.6716


Evaluating: 100%|██████████| 23/23 [00:07<00:00,  3.14it/s]


Validation Metrics: {'accuracy': 0.6994535519125683, 'perplexity': 1.754638788323857}
Saved best model!
Epoch 2/3


...

Validation Metrics: {'accuracy': 0.7062841530054644, 'perplexity': 1.7125813747635672}
Saved best model!
Epoch 3/3


Training: 100%|██████████| 69/69 [04:21<00:00,  3.79s/it]


Training Loss: 0.5151


Evaluating: 100%|██████████| 23/23 [00:06<00:00,  3.39it/s]


Validation Metrics: {'accuracy': 0.7144808743169399, 'perplexity': 1.6870675952140544}
Saved best model!
Training Complete!


In [18]:
# Load the best model
trained_gpt2_model.load_state_dict(torch.load("best_gpt2_nsp_model.pt"))

# Evaluate on the test set
test_metrics = evaluate_gpt2_model(trained_gpt2_model, test_dataloader)
print(f"Evaluation Results for Trained GPT-2 Model: {test_metrics}")

# Clear GPU memory
torch.cuda.empty_cache()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_356\1335959579.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_gpt2_model.load_state_dict(torch.load("best_gpt2_nsp_

Evaluation Results for Trained GPT-2 Model: {'accuracy': 0.7406876790830945, 'perplexity': 1.6317820747772733}
